In [0]:
# !pip uninstall tensorflow
# !pip install tensorflow==2.0.0

In [0]:
%matplotlib inline
%load_ext tensorboard
# %tensorboard --logdir logs

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dense, Dropout



In [3]:
tf.__version__

'2.0.0'

# Image-Related Neural Networks

In [0]:
conv_model = Sequential([
    Input(shape=(400, 400, 3)),
    Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"),
    Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"),
    MaxPool2D(),
    Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"),
    Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"),
    MaxPool2D(),
    Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"),
    Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"),
    MaxPool2D(),
    Flatten(), 
    BatchNormalization(),
    Dense(20, activation="relu"), 
    Dropout(0.1), 
    Dense(15, activation="relu"), 
    Dropout(0.1), 
    Dense(1, activation="sigmoid")
])
conv_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 400, 400, 64)      1792      
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 400, 400, 64)      36928     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 200, 200, 64)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 200, 200, 128)     73856     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 200, 200, 128)     147584    
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 100, 100, 128)     0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 100, 100, 256)    

In [0]:
conv_model.layers[-2].kernel.shape

TensorShape([3, 3, 256, 256])

In [0]:
conv_model2 = Sequential([
    Input(shape=(400, 400, 3)),
    Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"),
    Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"),
    MaxPool2D(),
    Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"),
    Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"),
    MaxPool2D(),
    Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"),
    Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"),
    MaxPool2D(), 
    Conv2D(filters=3, kernel_size=3, padding="same", activation="relu")
])
conv_model2.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 400, 400, 64)      1792      
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 400, 400, 64)      36928     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 200, 200, 64)      0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 200, 200, 128)     73856     
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 200, 200, 128)     147584    
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 100, 100, 128)     0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 100, 100, 256)    

## Feeding data to NNs

* Split train, validation and test sets in advance
* "tf.data.Dataset().from_tensor_slices()" - creates batches from input (images) if no enough memory to save the whole batch can save only "names" of the input (images).

In [0]:
file_names = []
BASE_DIR = os.path.abspath("./") # convert relative path to full path 
os.listdir(BASE_DIR) # directories are separate classes e.g. "dogs", "cats"
for image_class in os.listdir(BASE_DIR):
  folder_name = BASE_DIR + "/" + image_class + "/"
  file_names.extend([BASE_DIR + file_name for file_name in os.listdir(folder_name)])

In [0]:
file_names = np.array(file_names)
classes = [] # list of classes for each input file from file_names

In [0]:
def read_data(file_name, label):
  result_file = tf.io.read_file(file_name.decode())
  # need to use tf api not sklearn or any other
  result_image = tf.image.decode_image(result_file)
  # result_image = tf.image.resize(result_image, (224, 224)) # it's better images to be resized in advanced
  result_image /= 255

  return (result_image, label)

In [0]:
BATCH_SIZE = 32
dataset = tf.data.Dataset().from_tensor_slices((file_names, classes)) # eager execution can see what we got
# dataset = dataset.mainterleave(read_data) # for async
dataset = dataset.map(read_data) # if "batch" is before "map" read_data will cache batch size at once
dataset = dataset.shuffle(len(file_names)) # buffer_size how much infront elemets to take so that it can shuffle # , reshuffle_each_iteration=True
dataset = dataset.batch(BATCH_SIZE) # order matters first shuffle than batch
dataset = dataset.repeat() # EPOCH_NUM 


In [0]:
for data, label in dataset:
  print(data)
  print(label)
  break


In [0]:
model.fit(dataset, steps_per_epoch=int(len(dataset) / BATCH_SIZE)) # tf doesn't know how big the dataset is so needs "steps_per_epoch"